Attribute analysis 

In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# 7 · Year-generalized panel + feature importance (uses ALL faults up to 2024)
#     Version: recent_8m vs recent_24m (replaces recent_12m)
# ─────────────────────────────────────────────────────────────────────────────
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import logging, math, sys, json, numpy as np, pandas as pd
# ─── Config ─────────────────────────────────────────────────────────────────
OUT_DIR = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/A_fault_model_with_health")
EXPECTED_LIFE_YEARS = 35.0
N_THREADS = 3
KEEP_VOLTAGES           = {22, 33}
FAULT_CSV = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/FAULT DATA/HT_fault_cable_info_processed_with_affected.csv")
CABLE_CSV = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/SWNO_MASTER_COMBINED_FULL_FINAL3.csv")

def norm_sw(s):
    return (s.astype(str).str.upper().str.strip()
             .str.replace(r"^(SWNO_|SWNO|SW|S)\s*", "", regex=True)
             .str.replace(r"\D+","",regex=True)
             .replace("", np.nan)).astype("Int64")

def v_to_num(v):
    try: return float(str(v).lower().replace("kv",""))
    except: return math.nan
fault=pd.read_csv(FAULT_CSV,parse_dates=["TIME_OUTAGE"],low_memory=False)

sw_col="TO_SWITCH" if "TO_SWITCH" in fault.columns else fault.columns[0]
fault["SWITCH_ID"]=norm_sw(fault[sw_col])
fault=fault.dropna(subset=["SWITCH_ID","TIME_OUTAGE"])

if "VOLTAGE" in fault.columns:
    fault["VNUM"]=fault["VOLTAGE"].apply(v_to_num)
    fault=fault[fault["VNUM"].isin(KEEP_VOLTAGES)]
fault=fault.drop_duplicates(["SWITCH_ID","TIME_OUTAGE"])
fault["YM"]=fault["TIME_OUTAGE"].dt.to_period("M")
cables=pd.read_csv(CABLE_CSV, low_memory=False).drop_duplicates("DESTINATION_SWITCH_ID")
today=pd.Timestamp.utcnow().tz_localize(None)

cab=cables.rename(columns={"DESTINATION_SWITCH_ID":"SWITCH_ID",
                           "MEASUREDLENGTH":"LENGTH_M",
                           "COMMISSIONEDDATE":"DATE_INSTALLED",
                           "NO_OF_SEGMENT":"SEGMENTS"})
cab["LENGTH_KM"]=pd.to_numeric(cab["LENGTH_M"],errors="coerce")/1000
cab["DATE_INSTALLED"]=(pd.to_datetime(cab["DATE_INSTALLED"],errors="coerce",utc=True)
                       .dt.tz_localize(None))
cyc=[c for c in cab.columns if c.startswith("CYCLE_Month_")]
var=[c for c in cab.columns if c.startswith("VAR_Month_")]
cab["cycle_pm"]=cab[cyc].mean(1)
cab["load_range_idx"]=cab[var].mean(1)/cab[var].median(1)


for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS",
          "VECLIB_MAXIMUM_THREADS","NUMEXPR_NUM_THREADS"]:
    os.environ[k] = str(N_THREADS)

(OUT_DIR / "CORR").mkdir(parents=True, exist_ok=True)

def log(msg, *args): print(msg % args if args else msg)

# ─── Preconditions ─────────────────────────────────────────────────────────
req_fault_cols = {"SWITCH_ID","TIME_OUTAGE"}
req_cab_cols   = {"SWITCH_ID","LENGTH_KM","SEGMENTS","cycle_pm","load_range_idx","DATE_INSTALLED"}
missing_f = req_fault_cols - set(globals().get("fault", pd.DataFrame()).columns)
missing_c = req_cab_cols   - set(globals().get("cab",   pd.DataFrame()).columns)
if missing_f or missing_c:
    raise RuntimeError(f"Missing columns — fault: {missing_f}, cab: {missing_c}")

# ─── Ensure TIME_OUTAGE is tz-aware UTC ────────────────────────────────────
fault["TIME_OUTAGE"] = pd.to_datetime(fault["TIME_OUTAGE"], errors="coerce", utc=True)

# Precompute inter-fault deltas (hours) once for MTBF
fault_sorted = fault.sort_values(["SWITCH_ID", "TIME_OUTAGE"]).copy()
fault_sorted["dt_hours"] = (
    fault_sorted.groupby("SWITCH_ID")["TIME_OUTAGE"]
    .diff()
    .dt.total_seconds()
    .div(3600)
)

# ─── Build per-switch, per-year panel (history up to Jan 1 of year y) ─────
min_y = int(fault["TIME_OUTAGE"].dt.year.min())
max_y = int(fault["TIME_OUTAGE"].dt.year.max())
years = list(range(max(min_y + 1, 2016), max_y + 1))

panel_rows = []
for y in years:
    B    = pd.Timestamp(f"{y}-01-01", tz="UTC")
    Bm8  = B - pd.DateOffset(months=8)   # ← 8-month window
    Bm24 = B - pd.DateOffset(years=2)

    # history ≤ B-1day (no leakage)
    hist = fault_sorted[fault_sorted["TIME_OUTAGE"] < B]

    # next-year target (fault occurs in year y)
    tgt  = (fault_sorted[fault_sorted["TIME_OUTAGE"].dt.year == y]
            .groupby("SWITCH_ID").size().rename("TARGET_FAULT")).astype(int)

    # counts in history
    hist_cnt  = hist.groupby("SWITCH_ID").size().rename("hist_faults_cum")
    recent8   = hist[hist["TIME_OUTAGE"] >= Bm8 ].groupby("SWITCH_ID").size().rename("recent_8m")
    recent24  = hist[hist["TIME_OUTAGE"] >= Bm24].groupby("SWITCH_ID").size().rename("recent_24m")
    last_time = hist.groupby("SWITCH_ID")["TIME_OUTAGE"].max().rename("last_fault_time")

    # MTBF (hours) in history up to B
    mtbf_h = (hist.groupby("SWITCH_ID")["dt_hours"].mean()
                  .rename("mtbf_hours"))

    feat = cab[[
        "SWITCH_ID","LENGTH_KM","SEGMENTS","cycle_pm","load_range_idx","DATE_INSTALLED"
    ]].copy()

    feat = (
        feat
        .merge(hist_cnt,   on="SWITCH_ID", how="left")
        .merge(recent8,    on="SWITCH_ID", how="left")   # ← recent_8m
        .merge(recent24,   on="SWITCH_ID", how="left")
        .merge(last_time,  on="SWITCH_ID", how="left")
        .merge(mtbf_h,     on="SWITCH_ID", how="left")
        .merge(tgt,        on="SWITCH_ID", how="left")
    )

    # fill counts
    feat[["hist_faults_cum","recent_8m","recent_24m","TARGET_FAULT"]] = (
        feat[["hist_faults_cum","recent_8m","recent_24m","TARGET_FAULT"]]
        .fillna(0).astype(int)
    )

    # age (years)
    B_naive = B.tz_convert(None)
    feat["DATE_INSTALLED"] = pd.to_datetime(feat["DATE_INSTALLED"], errors="coerce")
    age_years = (B_naive  - feat["DATE_INSTALLED"]).dt.days / 365.25
    feat["age_years"] = age_years / EXPECTED_LIFE_YEARS
    # feat["age_years"] = ((B_naive - feat["DATE_INSTALLED"]).dt.days / 365.25).clip(lower=0)
    feat['log1p_age_years'] = np.log1p(feat["age_years"].clip(lower=0))

    # time since last fault (days)
    feat["time_since_last_fault_days"]  = (B - feat["last_fault_time"]).dt.days
    feat.loc[feat["last_fault_time"].isna(), "time_since_last_fault_days"] = np.nan

    # numeric coercions
    feat["LENGTH_KM"] = pd.to_numeric(feat["LENGTH_KM"], errors="coerce")
    feat["SEGMENTS"]  = pd.to_numeric(feat["SEGMENTS"],  errors="coerce")

    # attributes (aligned with health score; NO per-km faults)
    # s: joints density (segments-1 per km)
    feat["joint_density"] = (feat["SEGMENTS"].fillna(0).clip(lower=1) - 1) / feat["LENGTH_KM"].replace(0, np.nan)
    # i: interruption risk via inverse MTBF (hours); larger => riskier
    feat["i_mtbf_inv"] = 1.0 / feat["mtbf_hours"].clip(lower=1)

    # age helpers
    feat["age_frac"] = feat["age_years"]
    feat["log1p_age_frac"] = np.log1p(feat["age_frac"].clip(lower=0))
    # log1p transforms for skewed attrs — keep ONLY length
    feat["log1p_LENGTH_KM"] = np.log1p(pd.to_numeric(feat["LENGTH_KM"], errors="coerce").clip(lower=0))

    feat["YEAR"] = y
    panel_rows.append(feat)

panel = pd.concat(panel_rows, ignore_index=True)

# ─── Clean NaNs/Infs ──────────────────────────────────────────────────────
num_cols = panel.select_dtypes(include=[np.number]).columns.tolist()
panel[num_cols] = panel[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0)

# ─── Drop any years beyond 2024 ───────────────────────────────────────────
panel = panel[panel["YEAR"] <= 2024].copy()

# ─── Feature set (use counts, NOT per-km; drop all banned log1p) ─────────
feat_cols = [
    "cycle_pm", "load_range_idx",
    "hist_faults_cum",
    "joint_density", "i_mtbf_inv",
    "log1p_LENGTH_KM",
    "recent_8m", "recent_24m", "log1p_hist_faults_cum","log1p_age_frac",
]
# keep log1p_hist_faults_cum only if it exists; otherwise create it
if "log1p_hist_faults_cum" not in panel.columns and "hist_faults_cum" in panel.columns:
    panel["log1p_hist_faults_cum"] = np.log1p(panel["hist_faults_cum"])
feat_cols = [c for c in feat_cols if c in panel.columns]

# ─── Split: train on 2016–2023, test on 2024 ─────────────────────────────
train_df = panel[panel["YEAR"] < 2024].copy()
test_df  = panel[panel["YEAR"] == 2024].copy()

Xtr = train_df[feat_cols].values
ytr = (train_df["TARGET_FAULT"] > 0).astype(int).values
Xte = test_df[feat_cols].values
yte = (test_df["TARGET_FAULT"]  > 0).astype(int).values

# ─── Train Random Forest ─────────────────────────────────────────────────
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=50,
    min_samples_leaf=25,
    class_weight="balanced_subsample",
    n_jobs=N_THREADS,
    random_state=42
).fit(Xtr, ytr)

# ─── Evaluate & choose best hold-out ─────────────────────────────────────
if len(np.unique(yte)) > 1:
    use_X, use_y, eval_year, note = Xte, yte, 2024, "test"
else:
    val_df = train_df[train_df["YEAR"] == 2023]
    yv     = (val_df["TARGET_FAULT"] > 0).astype(int).values
    if len(val_df) and len(np.unique(yv)) > 1:
        use_X, use_y, eval_year, note = val_df[feat_cols].values, yv, 2023, "val(2023)"
    else:
        use_X, use_y, eval_year, note = Xtr, ytr, int(train_df["YEAR"].min()), "train(caution)"

pred_proba = rf.predict_proba(use_X)[:,1]
auc = roc_auc_score(use_y, pred_proba)
log("[Generalized] Eval on %s year=%s  AUC=%.3f", note, eval_year, auc)

# ─── Compute & save importances ──────────────────────────────────────────
gini_imp = pd.Series(rf.feature_importances_, index=feat_cols, name="rf_gini_importance") \
             .sort_values(ascending=False)
perm = permutation_importance(
    rf, use_X, use_y, n_repeats=10, random_state=42, n_jobs=N_THREADS
)
perm_imp = pd.Series(perm.importances_mean, index=feat_cols, name="perm_importance_mean") \
             .sort_values(ascending=False)
perm_std = pd.Series(perm.importances_std, index=feat_cols, name="perm_importance_std") \
             .loc[perm_imp.index]

imp_table = pd.concat([gini_imp, perm_imp, perm_std], axis=1)
(OUT_DIR / "CORR" / "feature_importance_generalized.csv").write_text(
    imp_table.to_csv(index=True)
)

panel_out_cols = ["SWITCH_ID","YEAR","TARGET_FAULT",
                  "cycle_pm","load_range_idx","hist_faults_cum",
                  "joint_density","i_mtbf_inv","log1p_LENGTH_KM",
                  "recent_8m","recent_24m","log1p_hist_faults_cum","log1p_age_frac"]
panel_out = panel[[c for c in panel_out_cols if c in panel.columns]]
panel_out.to_csv(OUT_DIR / "CORR" / "panel_generalized_year_switch.csv", index=False)

log("Top-15 features by permutation importance:\n%s", perm_imp.head(15).to_string())
log("Saved: feature_importance_generalized.csv, panel_generalized_year_switch.csv")

# ─────────────────────────────────────────────────────────────────────────────
# Visuals: load panel, fit RF, and plot with customizable bins/styles
# ─────────────────────────────────────────────────────────────────────────────
OUT_DIR = Path(OUT_DIR)
panel_csv = OUT_DIR / "CORR" / "panel_generalized_year_switch.csv"
assert panel_csv.exists(), f"Missing {panel_csv}. Run the generalized panel block first."
panel = pd.read_csv(panel_csv)

panel = panel[panel["YEAR"] <= 2024].copy()
panel = panel.replace([np.inf, -np.inf], np.nan).fillna(0)

# Same feature set for viz
pref_feat_cols = [
    "cycle_pm", "load_range_idx",
    "hist_faults_cum",
    "joint_density", "i_mtbf_inv",
    "log1p_LENGTH_KM",
    "recent_8m", "recent_24m", "log1p_hist_faults_cum","log1p_age_frac"
]
feat_cols = [c for c in pref_feat_cols if c in panel.columns]
if not feat_cols:
    num = panel.select_dtypes(include=[np.number]).columns.tolist()
    for drop in ["TARGET_FAULT","YEAR"]:
        if drop in num: num.remove(drop)
    feat_cols = num

TRAIN_END = 2023
TEST_YEAR = 2024
train_df = panel[panel["YEAR"] <= TRAIN_END].copy()
test_df  = panel[panel["YEAR"] == TEST_YEAR].copy()

Xtr, ytr = train_df[feat_cols].values, (train_df["TARGET_FAULT"] > 0).astype(int).values
Xte, yte = test_df[feat_cols].values,  (test_df["TARGET_FAULT"]  > 0).astype(int).values

rf = RandomForestClassifier(
    n_estimators=400, max_depth=70, 
    min_samples_leaf=25,
    class_weight="balanced_subsample",
    n_jobs=N_THREADS, 
    random_state=42
).fit(Xtr, ytr)

p_eval = rf.predict_proba(Xte)[:,1]
auc    = roc_auc_score(yte, p_eval)
print(f"[viz] Eval year={TEST_YEAR}  AUROC={auc:.3f}")

perm = permutation_importance(rf, Xte, yte, n_repeats=20, random_state=42, n_jobs=N_THREADS)
perm_mean = pd.Series(perm.importances_mean, index=feat_cols).sort_values()

# ─── Binning & styling ────────────────────────────────────────────────────
bin_settings = {
    "recent_8m":          {"method": "integer"},   # ← integer bins for counts
    "recent_24m":         {"method": "integer"},
    "SEGMENTS":           {"method": "integer"},   # (only if present in panel)
    "log1p_LENGTH_KM":    {"method": "quantile", "q": 8},
    "time_since_last_fault_days": {"method": "equal_width", "bins": 7},
    "i_mtbf_inv":         {"method": "quantile", "q": 6},
}
style_settings = {
    "Actual event rate": {"marker": "o", "linestyle": "-",  "linewidth": 2},
    "Model probability": {"marker": "s", "linestyle": "--", "linewidth": 2},
}

top_feats = perm_mean.index[::-1][:6]
for feat in top_feats:
    s = test_df[feat].astype(float).replace([np.inf,-np.inf], np.nan)
    if feat == "time_since_last_fault_days":
        s = s.replace(0, np.nan)
    mask = s.notna()
    s, yv, pv = s[mask], yte[mask], p_eval[mask]

    cfg    = bin_settings.get(feat, {"method":"quantile","q":6})
    method = cfg["method"]

    if method == "integer":
        s_int = s.round().astype(int)
        grp = (
            pd.DataFrame({"feat":s_int,"y":yv,"p":pv})
              .groupby("feat")
              .agg(event_rate=("y","mean"), pred_mean=("p","mean"), count=("y","size"))
              .reset_index()
              .sort_values("feat")
        )
        x_vals = grp["feat"].values

    elif method == "equal_width":
        bins = cfg.get("bins", 5)
        b    = pd.cut(s, bins=bins)
        grp  = (
            pd.DataFrame({"feat":s,"y":yv,"p":pv,"bin":b})
              .groupby("bin")
              .agg(feat_mean=("feat","mean"), event_rate=("y","mean"),
                   pred_mean=("p","mean"), count=("y","size"))
              .reset_index(drop=True)
              .sort_values("feat_mean")
        )
        x_vals = grp["feat_mean"].values

    else:  # quantile
        q = cfg.get("q", 5)
        try:
            b = pd.qcut(s, q=q, duplicates="drop")
        except Exception:
            b = pd.cut(s, bins=q)
        grp = (
            pd.DataFrame({"feat":s,"y":yv,"p":pv,"bin":b})
              .groupby("bin")
              .agg(feat_mean=("feat","mean"), event_rate=("y","mean"),
                   pred_mean=("p","mean"), count=("y","size"))
              .reset_index(drop=True)
              .sort_values("feat_mean")
        )
        x_vals = grp["feat_mean"].values

    fig, ax = plt.subplots(figsize=(7.5, 4.5))
    ax.plot(x_vals, grp["event_rate"], label="Actual event rate",
            marker=style_settings["Actual event rate"]["marker"],
            linestyle=style_settings["Actual event rate"]["linestyle"],
            linewidth=style_settings["Actual event rate"]["linewidth"])
    ax.plot(x_vals, grp["pred_mean"], label="Model probability",
            marker=style_settings["Model probability"]["marker"],
            linestyle=style_settings["Model probability"]["linestyle"],
            linewidth=style_settings["Model probability"]["linewidth"])
    ax.set_xlabel(feat)
    ax.set_ylabel("Probability of any fault in year")
    ax.set_title(f"{feat} vs risk (binned, {TEST_YEAR})  [n={int(grp['count'].sum())}]")
    ax.legend(); ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(OUT_DIR / "CORR" / f"rel_{feat}_{TEST_YEAR}.png", dpi=180)
    plt.close(fig)

fig, ax = plt.subplots(figsize=(8, 0.4 * max(1, len(perm_mean)) + 1))
ax.barh(perm_mean.index, perm_mean.values)
ax.set_xlabel("Permutation importance (Δ AUROC)")
ax.set_title(f"Feature importance on {TEST_YEAR}")
fig.tight_layout()
fig.savefig(OUT_DIR / f"imp_permutation_{TEST_YEAR}.png", dpi=180)
plt.close(fig)

print("All plots saved.")


/tmp/ipykernel_73070/1300082723.py:41: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  fault["YM"]=fault["TIME_OUTAGE"].dt.to_period("M")


[Generalized] Eval on test year=2024  AUC=0.776
Top-15 features by permutation importance:
log1p_LENGTH_KM          0.062891
i_mtbf_inv               0.032813
recent_24m               0.026172
log1p_hist_faults_cum    0.025391
hist_faults_cum          0.018359
joint_density            0.016797
load_range_idx           0.013672
cycle_pm                 0.011328
recent_8m                0.007812
log1p_age_frac           0.005078
Saved: feature_importance_generalized.csv, panel_generalized_year_switch.csv
[viz] Eval year=2024  AUROC=0.776


/tmp/ipykernel_73070/1300082723.py:353: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_73070/1300082723.py:353: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_73070/1300082723.py:353: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_73070/1300082723.py:353: FutureWarning: The default of observed=False is deprecated and will be changed

All plots saved.


Change the classes

In [11]:
# ─────────────────────────────────────────────────────────────────────────────
# 7 · Year-generalized panel + feature importance (uses ALL faults up to 2024)
#     Version: recent_8m vs recent_24m (replaces recent_12m)
#     NOTE: Classes flipped → label 1 = NO FAULT, label 0 = FAULT
#           Metrics/plots are still reported for FAULT as the “positive” event.
# ─────────────────────────────────────────────────────────────────────────────
import os, math, json, sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# ─── Config ─────────────────────────────────────────────────────────────────
OUT_DIR = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/A_fault_model_with_health")
EXPECTED_LIFE_YEARS = 35.0
N_THREADS = 3
KEEP_VOLTAGES = {22, 33}
FAULT_CSV = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/FAULT DATA/HT_fault_cable_info_processed_with_affected.csv")
CABLE_CSV = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/SWNO_MASTER_COMBINED_FULL_FINAL4.csv")

# ─── Helpers ────────────────────────────────────────────────────────────────
def norm_sw(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.upper().str.strip()
             .str.replace(r"^(SWNO_|SWNO|SW|S)\s*", "", regex=True)
             .str.replace(r"\D+","",regex=True)
             .replace("", np.nan)).astype("Int64")

def v_to_num(v):
    try:
        return float(str(v).lower().replace("kv",""))
    except Exception:
        return math.nan

def log(msg, *args):  # simple printf-style logger
    print(msg % args if args else msg)

def proba_fault(model, X) -> np.ndarray:
    """
    Return P(FAULT) given labels are flipped (1=NO FAULT, 0=FAULT).
    """
    i_fault = int(np.where(model.classes_ == 0)[0][0])  # class '0' == FAULT
    return model.predict_proba(X)[:, i_fault]

# ─── Thread caps for reproducibility/perf ───────────────────────────────────
for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS",
          "VECLIB_MAXIMUM_THREADS","NUMEXPR_NUM_THREADS"]:
    os.environ[k] = str(N_THREADS)

(OUT_DIR / "CORR").mkdir(parents=True, exist_ok=True)

# ─── Load & normalize faults ────────────────────────────────────────────────
fault = pd.read_csv(FAULT_CSV, parse_dates=["TIME_OUTAGE"], low_memory=False)
sw_col = "TO_SWITCH" if "TO_SWITCH" in fault.columns else fault.columns[0]
fault["SWITCH_ID"] = norm_sw(fault[sw_col])
fault = fault.dropna(subset=["SWITCH_ID","TIME_OUTAGE"])

if "VOLTAGE" in fault.columns:
    fault["VNUM"] = fault["VOLTAGE"].apply(v_to_num)
    fault = fault[fault["VNUM"].isin(KEEP_VOLTAGES)]

fault = fault.drop_duplicates(["SWITCH_ID","TIME_OUTAGE"])
fault["TIME_OUTAGE"] = pd.to_datetime(fault["TIME_OUTAGE"], errors="coerce", utc=True)
fault["YM"] = fault["TIME_OUTAGE"].dt.to_period("M")

# ─── Load & normalize cable attributes ─────────────────────────────────────
cables = pd.read_csv(CABLE_CSV, low_memory=False).drop_duplicates("DESTINATION_SWITCH_ID")
cab = cables.rename(columns={
    "DESTINATION_SWITCH_ID": "SWITCH_ID",
    "MEASUREDLENGTH": "LENGTH_M",
    "COMMISSIONEDDATE": "DATE_INSTALLED",
    "NO_OF_SEGMENT": "SEGMENTS"
})
cab["LENGTH_KM"] = pd.to_numeric(cab["LENGTH_M"], errors="coerce")/1000
cab["DATE_INSTALLED"] = (pd.to_datetime(cab["DATE_INSTALLED"], errors="coerce", utc=True)
                         .dt.tz_localize(None))

cyc = [c for c in cab.columns if c.startswith("CYCLE_Month_")]
var = [c for c in cab.columns if c.startswith("VAR_Month_")]
cab["cycle_pm"] = cab[cyc].mean(1) if len(cyc) else 0.0
cab["load_range_idx"] = (cab[var].mean(1)/cab[var].median(1)) if len(var) else 0.0

# ─── Preconditions ─────────────────────────────────────────────────────────
req_fault_cols = {"SWITCH_ID","TIME_OUTAGE"}
req_cab_cols   = {"SWITCH_ID","LENGTH_KM","SEGMENTS","cycle_pm","load_range_idx","DATE_INSTALLED"}
missing_f = req_fault_cols - set(fault.columns)
missing_c = req_cab_cols   - set(cab.columns)
if missing_f or missing_c:
    raise RuntimeError(f"Missing columns — fault: {missing_f}, cab: {missing_c}")

# ─── Precompute inter-fault deltas (hours) once for MTBF ───────────────────
fault_sorted = fault.sort_values(["SWITCH_ID", "TIME_OUTAGE"]).copy()
fault_sorted["dt_hours"] = (
    fault_sorted.groupby("SWITCH_ID")["TIME_OUTAGE"]
    .diff()
    .dt.total_seconds()
    .div(3600)
)

# ─── Build per-switch, per-year panel (history up to Jan 1 of year y) ──────
min_y = int(fault["TIME_OUTAGE"].dt.year.min())
max_y = int(fault["TIME_OUTAGE"].dt.year.max())
years = list(range(max(min_y + 1, 2016), max_y + 1))

panel_rows = []
for y in years:
    B    = pd.Timestamp(f"{y}-01-01", tz="UTC")
    Bm8  = B - pd.DateOffset(months=8)    # 8-month recency window
    Bm24 = B - pd.DateOffset(years=2)     # 24-month recency window

    # history ≤ B-1day (no leakage)
    hist = fault_sorted[fault_sorted["TIME_OUTAGE"] < B]

    # next-year target (fault occurs in year y)
    tgt = (fault_sorted[fault_sorted["TIME_OUTAGE"].dt.year == y]
           .groupby("SWITCH_ID").size().rename("TARGET_FAULT")).astype(int)

    # counts in history
    hist_cnt  = hist.groupby("SWITCH_ID").size().rename("hist_faults_cum")
    recent8   = hist[hist["TIME_OUTAGE"] >= Bm8 ].groupby("SWITCH_ID").size().rename("recent_8m")
    recent24  = hist[hist["TIME_OUTAGE"] >= Bm24].groupby("SWITCH_ID").size().rename("recent_24m")
    last_time = hist.groupby("SWITCH_ID")["TIME_OUTAGE"].max().rename("last_fault_time")
    mtbf_h    = (hist.groupby("SWITCH_ID")["dt_hours"].mean().rename("mtbf_hours"))

    feat = cab[["SWITCH_ID","LENGTH_KM","SEGMENTS","cycle_pm","load_range_idx","DATE_INSTALLED"]].copy()
    feat = (
        feat
        .merge(hist_cnt,   on="SWITCH_ID", how="left")
        .merge(recent8,    on="SWITCH_ID", how="left")
        .merge(recent24,   on="SWITCH_ID", how="left")
        .merge(last_time,  on="SWITCH_ID", how="left")
        .merge(mtbf_h,     on="SWITCH_ID", how="left")
        .merge(tgt,        on="SWITCH_ID", how="left")
    )

    # fill counts
    for c in ["hist_faults_cum","recent_8m","recent_24m","TARGET_FAULT"]:
        if c in feat.columns:
            feat[c] = feat[c].fillna(0).astype(int)

    # age (years, fractional vs EXPECTED_LIFE_YEARS)
    B_naive = B.tz_convert(None)
    feat["DATE_INSTALLED"] = pd.to_datetime(feat["DATE_INSTALLED"], errors="coerce")
    age_years = (B_naive - feat["DATE_INSTALLED"]).dt.days / 365.25
    feat["age_years"] = age_years / EXPECTED_LIFE_YEARS
    feat["log1p_age_years"] = np.log1p(feat["age_years"].clip(lower=0))

    # time since last fault (days)
    feat["time_since_last_fault_days"] = (B - feat["last_fault_time"]).dt.days
    feat.loc[feat["last_fault_time"].isna(), "time_since_last_fault_days"] = np.nan

    # numeric coercions
    feat["LENGTH_KM"] = pd.to_numeric(feat["LENGTH_KM"], errors="coerce")
    feat["SEGMENTS"]  = pd.to_numeric(feat["SEGMENTS"],  errors="coerce")

    # attributes (aligned with health score; NOT per-km faults)
    feat["joint_density"] = (feat["SEGMENTS"].fillna(0).clip(lower=1) - 1) / feat["LENGTH_KM"].replace(0, np.nan)
    feat["i_mtbf_inv"]    = 1.0 / feat["mtbf_hours"].clip(lower=1)

    feat["age_frac"]        = feat["age_years"]
    feat["log1p_age_frac"]  = np.log1p(feat["age_frac"].clip(lower=0))
    feat["log1p_LENGTH_KM"] = np.log1p(pd.to_numeric(feat["LENGTH_KM"], errors="coerce").clip(lower=0))

    feat["YEAR"] = y
    panel_rows.append(feat)

panel = pd.concat(panel_rows, ignore_index=True)

# Clean NaNs/Infs
num_cols = panel.select_dtypes(include=[np.number]).columns.tolist()
panel[num_cols] = panel[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop any years beyond 2024
panel = panel[panel["YEAR"] <= 2024].copy()

# Feature set (counts + attrs)
feat_cols = [
    "cycle_pm", "load_range_idx",
    "hist_faults_cum",
    "joint_density", "i_mtbf_inv",
    "log1p_LENGTH_KM",
    "recent_8m", "recent_24m", "log1p_hist_faults_cum","log1p_age_frac",
]
# keep / synthesize log1p_hist_faults_cum
if "log1p_hist_faults_cum" not in panel.columns and "hist_faults_cum" in panel.columns:
    panel["log1p_hist_faults_cum"] = np.log1p(panel["hist_faults_cum"])
feat_cols = [c for c in feat_cols if c in panel.columns]

# ─── Split: train on 2016–2023, eval on 2024 (with fallback) ───────────────
train_df = panel[panel["YEAR"] < 2024].copy()
test_df  = panel[panel["YEAR"] == 2024].copy()

# ↻ FLIPPED CLASSES: positive=NO FAULT (1), negative=FAULT (0)
Xtr = train_df[feat_cols].values
ytr = (train_df["TARGET_FAULT"] == 0).astype(int).values
Xte = test_df[feat_cols].values
yte = (test_df["TARGET_FAULT"]  == 0).astype(int).values

# Train RF
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=50,
    min_samples_leaf=25,
    class_weight="balanced_subsample",
    n_jobs=N_THREADS,
    random_state=42
).fit(Xtr, ytr)

# Choose best holdout for reporting AUROC with FAULT as positive
if len(np.unique(yte)) > 1 and len(yte):
    use_X, use_y, eval_year, note = Xte, yte, 2024, "test"
else:
    val_df = train_df[train_df["YEAR"] == 2023]
    yv = (val_df["TARGET_FAULT"] == 0).astype(int).values
    if len(val_df) and len(np.unique(yv)) > 1:
        use_X, use_y, eval_year, note = val_df[feat_cols].values, yv, 2023, "val(2023)"
    else:
        use_X, use_y, eval_year, note = Xtr, ytr, int(train_df["YEAR"].min()), "train(caution)"

# Evaluate AUROC treating FAULT as the positive event
p_fault = proba_fault(rf, use_X)      # P(FAULT)
y_fault = 1 - use_y                   # convert labels so 1=FAULT for metrics
auc = roc_auc_score(y_fault, p_fault)
log("[Generalized] Eval on %s year=%s  AUROC(fault+)=%.3f", note, eval_year, auc)

# ─── Feature importances (gini + permutation on chosen holdout) ────────────
gini_imp = pd.Series(rf.feature_importances_, index=feat_cols, name="rf_gini_importance") \
             .sort_values(ascending=False)
perm = permutation_importance(
    rf, use_X, use_y, n_repeats=10, random_state=42, n_jobs=N_THREADS
)
perm_imp = pd.Series(perm.importances_mean, index=feat_cols, name="perm_importance_mean") \
             .sort_values(ascending=False)
perm_std = pd.Series(perm.importances_std, index=feat_cols, name="perm_importance_std") \
             .loc[perm_imp.index]

imp_table = pd.concat([gini_imp, perm_imp, perm_std], axis=1)
(OUT_DIR / "CORR" / "feature_importance_generalized.csv").write_text(
    imp_table.to_csv(index=True)
)

# Persist compact panel (for viz step below)
panel_out_cols = ["SWITCH_ID","YEAR","TARGET_FAULT",
                  "cycle_pm","load_range_idx","hist_faults_cum",
                  "joint_density","i_mtbf_inv","log1p_LENGTH_KM",
                  "recent_8m","recent_24m","log1p_hist_faults_cum","log1p_age_frac"]
panel_out = panel[[c for c in panel_out_cols if c in panel.columns]]
panel_out.to_csv(OUT_DIR / "CORR" / "panel_generalized_year_switch.csv", index=False)

log("Top-15 features by permutation importance:\n%s", perm_imp.head(15).to_string())
log("Saved: feature_importance_generalized.csv, panel_generalized_year_switch.csv")

# ─────────────────────────────────────────────────────────────────────────────
# Visuals: load panel, fit RF, and plot with customizable bins/styles
# ─────────────────────────────────────────────────────────────────────────────
panel_csv = OUT_DIR / "CORR" / "panel_generalized_year_switch.csv"
assert panel_csv.exists(), f"Missing {panel_csv}. Run the generalized panel block first."
panel = pd.read_csv(panel_csv)
panel = panel[panel["YEAR"] <= 2024].copy()
panel = panel.replace([np.inf, -np.inf], np.nan).fillna(0)

# Same feature set for viz
pref_feat_cols = [
    "cycle_pm", "load_range_idx",
    "hist_faults_cum",
    "joint_density", "i_mtbf_inv",
    "log1p_LENGTH_KM",
    "recent_8m", "recent_24m", "log1p_hist_faults_cum","log1p_age_frac"
]
feat_cols = [c for c in pref_feat_cols if c in panel.columns]
if not feat_cols:
    num = panel.select_dtypes(include=[np.number]).columns.tolist()
    for drop in ["TARGET_FAULT","YEAR"]:
        if drop in num: num.remove(drop)
    feat_cols = num

TRAIN_END = 2023
TEST_YEAR = 2024
train_df = panel[panel["YEAR"] <= TRAIN_END].copy()
test_df  = panel[panel["YEAR"] == TEST_YEAR].copy()

# ↻ FLIPPED CLASSES again for viz model
Xtr, ytr = train_df[feat_cols].values, (train_df["TARGET_FAULT"] == 0).astype(int).values
Xte, yte = test_df[feat_cols].values,  (test_df["TARGET_FAULT"]  == 0).astype(int).values

rf = RandomForestClassifier(
    n_estimators=300, max_depth=50,
    min_samples_leaf=25,
    class_weight="balanced_subsample",
    n_jobs=N_THREADS, random_state=42
).fit(Xtr, ytr)

# Evaluate on test with FAULT-as-positive AUROC
p_eval_fault = proba_fault(rf, Xte)   # P(FAULT)
yte_fault = 1 - yte                   # 1=FAULT for metrics/plots
auc = roc_auc_score(yte_fault, p_eval_fault)
print(f"[viz] Eval year={TEST_YEAR}  AUROC(fault+)= {auc:.3f}")

# Permutation importance (default scoring = accuracy)
perm = permutation_importance(rf, Xte, yte, n_repeats=20, random_state=42, n_jobs=N_THREADS)
perm_mean = pd.Series(perm.importances_mean, index=feat_cols).sort_values()

# ─── Binning & styling ──────────────────────────────────────────────────────
bin_settings = {
    "recent_8m":          {"method": "integer"},   # integer bins for counts
    "recent_24m":         {"method": "integer"},
    "SEGMENTS":           {"method": "integer"},   # (if present)
    "log1p_LENGTH_KM":    {"method": "quantile", "q": 8},
    "time_since_last_fault_days": {"method": "equal_width", "bins": 7},
    "i_mtbf_inv":         {"method": "quantile", "q": 6},
}
style_settings = {
    "Actual fault rate": {"marker": "o", "linestyle": "-",  "linewidth": 2},
    "Predicted fault prob.": {"marker": "s", "linestyle": "--", "linewidth": 2},
}

top_feats = perm_mean.index[::-1][:6]
for feat in top_feats:
    if feat not in test_df.columns:
        continue
    s = test_df[feat].astype(float).replace([np.inf,-np.inf], np.nan)
    if feat == "time_since_last_fault_days":
        s = s.replace(0, np.nan)
    mask = s.notna()

    # Use FAULT as positive (1) for calibration curves
    yv = yte_fault[mask]          # 1 = FAULT
    pv = p_eval_fault[mask]       # P(FAULT)
    s  = s[mask]

    cfg    = bin_settings.get(feat, {"method":"quantile","q":6})
    method = cfg["method"]

    if method == "integer":
        s_int = s.round().astype(int)
        grp = (
            pd.DataFrame({"feat":s_int,"y":yv,"p":pv})
              .groupby("feat")
              .agg(event_rate=("y","mean"), pred_mean=("p","mean"), count=("y","size"))
              .reset_index()
              .sort_values("feat")
        )
        x_vals = grp["feat"].values

    elif method == "equal_width":
        bins = cfg.get("bins", 5)
        b    = pd.cut(s, bins=bins)
        grp  = (
            pd.DataFrame({"feat":s,"y":yv,"p":pv,"bin":b})
              .groupby("bin")
              .agg(feat_mean=("feat","mean"), event_rate=("y","mean"),
                   pred_mean=("p","mean"), count=("y","size"))
              .reset_index(drop=True)
              .sort_values("feat_mean")
        )
        x_vals = grp["feat_mean"].values

    else:  # quantile
        q = cfg.get("q", 5)
        try:
            b = pd.qcut(s, q=q, duplicates="drop")
        except Exception:
            b = pd.cut(s, bins=q)
        grp = (
            pd.DataFrame({"feat":s,"y":yv,"p":pv,"bin":b})
              .groupby("bin")
              .agg(feat_mean=("feat","mean"), event_rate=("y","mean"),
                   pred_mean=("p","mean"), count=("y","size"))
              .reset_index(drop=True)
              .sort_values("feat_mean")
        )
        x_vals = grp["feat_mean"].values

    fig, ax = plt.subplots(figsize=(7.5, 4.5))
    ax.plot(x_vals, grp["event_rate"], label="Actual fault rate",
            marker=style_settings["Actual fault rate"]["marker"],
            linestyle=style_settings["Actual fault rate"]["linestyle"],
            linewidth=style_settings["Actual fault rate"]["linewidth"])
    ax.plot(x_vals, grp["pred_mean"], label="Predicted fault prob.",
            marker=style_settings["Predicted fault prob."]["marker"],
            linestyle=style_settings["Predicted fault prob."]["linestyle"],
            linewidth=style_settings["Predicted fault prob."]["linewidth"])
    ax.set_xlabel(feat)
    ax.set_ylabel("Probability of any fault in year")
    ax.set_title(f"{feat} vs risk (binned, {TEST_YEAR})  [n={int(grp['count'].sum())}]")
    ax.legend(); ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(OUT_DIR / "CORR" / f"rel_{feat}_{TEST_YEAR}.png", dpi=180)
    plt.close(fig)

# Permutation-importance bar (holds default scoring semantics)
fig, ax = plt.subplots(figsize=(8, 0.4 * max(1, len(perm_mean)) + 1))
ax.barh(perm_mean.index, perm_mean.values)
ax.set_xlabel("Permutation importance (Δ accuracy)")
ax.set_title(f"Feature importance on {TEST_YEAR}")
fig.tight_layout()
fig.savefig(OUT_DIR / f"imp_permutation_{TEST_YEAR}.png", dpi=180)
plt.close(fig)

print("All plots saved.")


/tmp/ipykernel_16463/3881479799.py:67: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  fault["YM"] = fault["TIME_OUTAGE"].dt.to_period("M")


[Generalized] Eval on test year=2024  AUROC(fault+)=0.771
Top-15 features by permutation importance:
log1p_LENGTH_KM          0.055469
recent_24m               0.028516
i_mtbf_inv               0.026953
hist_faults_cum          0.022266
log1p_hist_faults_cum    0.019922
joint_density            0.017188
load_range_idx           0.012109
log1p_age_frac           0.010937
cycle_pm                 0.010547
recent_8m                0.004687
Saved: feature_importance_generalized.csv, panel_generalized_year_switch.csv
[viz] Eval year=2024  AUROC(fault+)= 0.771


/tmp/ipykernel_16463/3881479799.py:369: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_16463/3881479799.py:369: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_16463/3881479799.py:369: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("bin")
/tmp/ipykernel_16463/3881479799.py:369: FutureWarning: The default of observed=False is deprecated and will be changed

All plots saved.


In [2]:
# ----- Make W proportional to permutation importance -----

# 1) Map model features to health-score factors
feat2factor = {
    "cycle_pm":                "c",
    "load_range_idx":          "r",
    "log1p_age_frac":          "a",
    "age_frac":                "a",
    "age_years":               "a",

    "log1p_hist_faults_cum":   "f",
    "hist_faults_cum":         "f",

    "joint_density":           "s",

    "i_mtbf_inv":              "i",

    "log1p_LENGTH_KM":         "l",

    # u (recent faults): prefer 24m, fall back to 12m if present
    "recent_24m":              "u",
    "recent_12m":              "u",
}

# 2) Collect permutation importance into factors
#    (perm_mean is a Series: index=feature names, values=ΔAUROC)
factor_scores = {k: 0.0 for k in list("crasifulu")}  # c r a f s i l u (order doesn’t matter)
for feat, imp in perm_mean.items():
    f = feat2factor.get(feat)
    if f is not None:
        factor_scores[f] += max(0.0, float(imp))  # guard against tiny negative noise

# 3) If some factors are 0 (e.g., feature missing), keep them at 0.
#    Normalize the eight factors to sum to 0.85; keep p fixed at 0.15.
sum_imp = sum(factor_scores.values())
if sum_imp <= 0:
    # fallback: split 0.85 equally if everything is zero
    per = 0.85 / 8.0
    for k in factor_scores: factor_scores[k] = per
else:
    for k in factor_scores:
        factor_scores[k] = 0.85 * (factor_scores[k] / sum_imp)

W = dict(**factor_scores, p=0.15)  # add p term

# 4) (Optional) Pretty print + sanity checks
print("Data-driven weights (sum should be 1.0):")
for k in sorted(W.keys()):
    print(f"  {k}: {W[k]:.5f}")
print("Sum =", round(sum(W.values()), 6))


Data-driven weights (sum should be 1.0):
  a: 0.01777
  c: 0.04559
  f: 0.17541
  i: 0.12673
  l: 0.24495
  p: 0.15000
  r: 0.05332
  s: 0.07495
  u: 0.11127
Sum = 1.0
